In [6]:
import pandas as pd

data = pd.read_csv("/Users/csb/Desktop/Project/넙치 머신러닝/양식어류 질병예측 모델 최종본/10Feature.txt",sep='\t')
data

,Temperature,Condition factor,T P,Wgp2,Wgi3,Whkp2,Whki2-1,Wlr1,Wlr2-1,Wsi2,CLASS
0,12.44,1.828,5.7,0,4,0,0,0,0,8,Warning
1,12.44,2.348,5.4,0,0,8,0,4,0,4,Warning
2,12.44,1.634,4.7,0,8,0,8,4,0,8,Infection
3,12.44,1.906,5.7,4,4,0,8,4,0,4,Warning
4,12.44,2.055,5.9,4,4,4,4,0,0,0,Warning
...,...,...,...,...,...,...,...,...,...,...,...
1211,20.90,1.672,3.1,0,0,0,0,0,0,12,Warning
1212,20.90,2.115,3.3,0,0,0,0,4,0,12,Warning
1213,20.90,2.122,3.4,0,0,0,8,12,4,12,Warning
1214,20.90,2.070,4.2,0,0,0,0,0,0,12,Normal


In [7]:
import sys
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pickle

data.head()

selectedFeatures_ofStat = ["Temperature", "Condition factor", "T P", "Wgp2", "Wgi3", "Whkp2",
                           "Whki2-1", "Wlr1", "Wlr2-1", "Wsi2", "CLASS"]

data = data.loc[:,selectedFeatures_ofStat]
data = pd.DataFrame.dropna(data, axis=0)

mapping_dict = {'Normal' : 1, 'Warning' : 2, 'Infection' : 3}
after_mapping_target = data['CLASS'].apply(lambda x : mapping_dict[x])

feature_columns = list(data.columns.difference(['CLASS']))
X = data[feature_columns]
y = after_mapping_target

model = RandomForestClassifier(n_estimators=100, max_depth=20, random_state=1234)
model.fit(X,y)

pickle.dump(model, open('10featureModel.pkl', 'wb'))

In [11]:
import sys
import pandas as pd
import pickle

with open("/Users/csb/Desktop/Project/넙치 머신러닝/양식어류 질병예측 모델 최종본/10featureModel.pkl", 'rb') as file:
    model = pickle.load(file)

testFile_path = sys.argv[1]

data = pd.read_csv("/Users/csb/Desktop/Project/넙치 머신러닝/양식어류 질병예측 모델 최종본/10featureSampledata.txt", sep="\t")
selectedFeatures_ofStat = ["ID","Temperature", "Condition factor", "T P", "Wgp2", "Wgi3", "Whkp2",
                           "Whki2-1", "Wlr1", "Wlr2-1", "Wsi2"]
data = data.loc[:,selectedFeatures_ofStat]
data = pd.DataFrame.dropna(data, axis=0)

feature_columns = list(data.columns.difference(['ID']))
idCol = data.loc[:,"ID"]
test_x = data[feature_columns]

prediction = model.predict(test_x)

mapping_dict = {1 : 'Normal', 2 : 'Warning', 3 : 'Infection'}
prediction = [mapping_dict[index] for index in prediction]

result_df = pd.DataFrame({'ID': idCol, 'Prediction': prediction})

result_df.to_csv('10featureOutput.txt', sep='\t', index=False)

file.close()